In [22]:
from pathlib import Path
import re

In [23]:
import hashlib
from difflib import unified_diff


# go through all folders and files in ./comparison/old and for each file
# find the corresponding file in ./comparison/new and compare the two files
def compare_dirs(dir_one: Path, dir_two: Path, print_diff: bool = False):
    for old_file in old_dir.rglob("**/*"):
        if old_file.suffix in [".txt", ".xml"]:
            new_file = new_dir / old_file.relative_to(old_dir)
            if new_file.exists():
                with open(old_file, "r") as f:
                    old_content = f.read()
                with open(new_file, "r") as f:
                    new_content = f.read()

                # Calculate the hash of the old content
                old_hash = hashlib.md5(old_content.encode()).hexdigest()

                # Calculate the hash of the new content
                new_hash = hashlib.md5(new_content.encode()).hexdigest()

                # Compare the hashes
                if old_hash != new_hash:
                    print(f"Files {old_file} and {new_file} differ.")

                    # show difference with difflib
                    if print_diff:
                        for line in unified_diff(
                            old_content.splitlines(),
                            new_content.splitlines(),
                            lineterm="",
                        ):
                            print(line)

            else:
                print(f"File {new_file} does not exist.")

In [25]:
old_dir = Path("../comparison/old")
new_dir = Path("../comparison/new")

compare_dirs(old_dir, new_dir, print_diff=True)

In [ ]:
# Deletes all the newlines in brackets
import regex


def f1(filetext):
    bracket_text = regex.finditer(r"\(([^(\)]*(\(([^(\)]*)\))*[^(\)]*)\)", filetext)

    for bracket in bracket_text:
        filetext = filetext.replace(
            str(bracket.group()),
            regex.sub(
                r"\n+",
                " ",
                regex.sub(
                    r"(^((?<!Abg\.).)+|^.*\[.+)(-\n+)",
                    r"\1",
                    str(bracket.group()),
                    flags=regex.MULTILINE,
                ),
            ),
        )

    return filetext


def f2(filetext: str) -> str:
    # Finds all text within parentheses, including nested parentheses
    bracket_text = regex.findall(r"\(([^(\)]*(\(([^(\)]*)\))*[^(\)]*)\)", filetext)

    for sub_groups in bracket_text:
        # findall does not return the outer brackets compared to finditer
        bracket_content = sub_groups[0]
        bracket_content = regex.sub(
            r"(^((?<!Abg\.).)+|^.*\[.+)(-\n+)",
            r"\1",
            bracket_content,
            flags=regex.MULTILINE,
        )
        bracket_content = regex.sub(r"\n+", " ", bracket_content)
        filetext = filetext.replace(sub_groups[0], bracket_content)

    return filetext


def f3(filetext: str) -> str:
    bracket_text = regex.finditer(r"\(([^(\)]*(\(([^(\)]*)\))*[^(\)]*)\)", filetext)

    for bracket in bracket_text:
        bracket_text = bracket.group()
        # Remove all dashes followed by a new line
        bracket_text = regex.sub(
            r"(^((?<!Abg\.).)+|^.*\[.+)(-\n+)",
            r"\1",
            bracket_text,
            flags=regex.MULTILINE,
        )
        # Replace newline with spaces
        bracket_text = regex.sub(r"\n+", " ", bracket_text)
        # Replace the original text with the cleaned text
        filetext = filetext.replace(bracket.group(), bracket_text)

    return filetext

In [46]:
text = """Ohne den Sinn des Atlantikpakts zu ändern, wäre es nicht möglich, daß Staaten dem Pakt beitreten, die territoriale Ansprüche vorzubringen haben. „Das würde für uns das Risiko bedeuten, in einen Angriffskrieg verwickelt zu werden." —
(Hört! Hört! bei der SPD und auf der
äußersten Rechten. — Zuruf des Abg. Dr.
Bucerius: Er erkennt also unsere Ansprüche an!)
Das ist Punkt zwei."""

print(repr(f1(text)))

print(repr(f3(text)))

f1(text) == f3(text)


'Ohne den Sinn des Atlantikpakts zu ändern, wäre es nicht möglich, daß Staaten dem Pakt beitreten, die territoriale Ansprüche vorzubringen haben. „Das würde für uns das Risiko bedeuten, in einen Angriffskrieg verwickelt zu werden." —\n(Hört! Hört! bei der SPD und auf der äußersten Rechten. — Zuruf des Abg. Dr. Bucerius: Er erkennt also unsere Ansprüche an!)\nDas ist Punkt zwei.'
'Ohne den Sinn des Atlantikpakts zu ändern, wäre es nicht möglich, daß Staaten dem Pakt beitreten, die territoriale Ansprüche vorzubringen haben. „Das würde für uns das Risiko bedeuten, in einen Angriffskrieg verwickelt zu werden." —\n(Hört! Hört! bei der SPD und auf der äußersten Rechten. — Zuruf des Abg. Dr. Bucerius: Er erkennt also unsere Ansprüche an!)\nDas ist Punkt zwei.'


True

In [38]:
%%timeit

# demonstrate re.findall
text = "This is a (test (string) with (nested) parentheses) and (more) text."
bracket_text = re.findall(r"\(([^(\)]*(\(([^(\)]*)\))*[^(\)]*)\)", text)


1.59 μs ± 120 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [37]:
%%timeit

# demonstrate re.findall
text = "This is a (test (string) with (nested) parentheses) and (more) text."
bracket_text = re.finditer(r"\(([^(\)]*(\(([^(\)]*)\))*[^(\)]*)\)", text)


361 ns ± 6.09 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [43]:
text = "This is a (test (string) with (nested) parentheses) and (more) text."
bracket_text = re.finditer(r"\(([^(\)]*(\(([^(\)]*)\))*[^(\)]*)\)", text)
